# Effective Python

This notebook contains the best practices taken from the homonimous book.

Only some chapters are considered here, purely based on the author's ignorance.

In [1]:
import sys
print(sys.version)

3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]


## It's good to unpack multiple varibles 
...rather than indexing.

Use **f-strings** to display the output.


In [2]:
def out(i):
    assert i > 0
    return list(range(i))

first, second, *others = out(10)
print(f"{first:0.3f}, {second} and {others}")

0.000, 1 and [2, 3, 4, 5, 6, 7, 8, 9]


## The following evaluate to `False`

Recall these niceties about empty strings, empty lists and `0`:

In [3]:
if []:
    print("Not printing")
elif "":
    print("Not printing")
elif 0:
    print("Not printing")
else:
    print("Got it?")

Got it?


## RAM manipulations

Direct manipulation of the memory cells.

In [4]:
a = b'123456'  # byte expression
aa = "sdojfbh"  # string expression
mv = memoryview(a)
try:
    memoryview(aa)
except TypeError:
    print("Only bytes can be manipulated in memory, not strings")
mv

Only bytes can be manipulated in memory, not strings


In [5]:
mv[2:5].tobytes()  # selecting a subsection of the RAM


b'345'

## Use ternary expression to simplify your code

In [6]:
def true_or_false(input):
    """This function will print what the
    input will evaluate to."""
    b = "True" if input else "False"
    print(b)
    
true_or_false([])

False


## Striding 
...and revertng lists.

In [7]:
## Striding
start = 1
end = 10
step = 3

test = list(range(end + 7))

print(f"Full list: {test}, \nStrided list: {test[start:end:step]}")

# A nice trick to reverse the list:
test[::-1]

Full list: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 
Strided list: [1, 4, 7]


[16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

## Sorting objects with lambda functions

Check out the abstract class and how the lambda is used for sorting (and also notice the use of `__repr__`).

In [8]:
class Obj():
    """An object

    Args:
        name (str):
            the name of the Obj
        height (float):
            the heght of the Obj
    """
    def __init__(self, name, height):
        self.name = name
        self.height = height
        
    def __repr__(self):
        """This method is used by the print function.
        It needs to return a string"""
        return f"({self.name}, {self.height})"

    def method(self):
        """whatever, not needed"""
        pass


ob1, ob2, ob3 = Obj("a", 1), Obj("bbb", 3), Obj("c", 2)

list_of_obj = [ob1, ob2, ob3]
try:
    list_of_obj.sort()
except TypeError:
    print("The sorting is not a priori defined!")

list_of_obj.sort(key=lambda x:x.name)  # we are telling to sort by the name
print(f"Sort by name: {list_of_obj}")

list_of_obj.sort(key=lambda x:x.height)  # we are telling to sort by the height
print(f"Sort by name: {list_of_obj}")

The sorting is not a priori defined!
Sort by name: [(a, 1), (bbb, 3), (c, 2)]
Sort by name: [(a, 1), (c, 2), (bbb, 3)]


## Add `__missing__` to handle missing keys in `dict`

In [9]:
class NewDict(dict):
    def __missing__(self, key):
        """This special method is called when the key
        is not defined yet."""
        self[key] = 0.0

d = NewDict()
d["key1"] = 5
d["key2"]  # this is not defined, hence filled in with `0.0`
d

{'key1': 5, 'key2': 0.0}

## Use of coroutines to parallelise

These are functions that are run asyn and stop when the await step is reached.

In [10]:
try:
    import asyncio

    obj = []

    async def func(i):  # this is a coroutine
        print(f'I am doing something {i}')
        obj.append(i)

    async def manager():  # this is a coroutine
        tasks = []
        for i in range(10):
            task = func(i)  # Fan out
            tasks.append(task)
        await asyncio.gather(*tasks)  # Fan in
        return obj

    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    task = loop.create_task(manager())
    new_obj = loop.run_until_complete(task)
    print(new_obj)
except RuntimeError:
    print("Jupyter notebook is already running as a coroutine, hence it is ot possible to run another one on the same thread")

Jupyter notebook is already running as a coroutine, hence it is ot possible to run another one on the same thread


## Closure

Variables inside the scope of a function are not modified for outer scopes.

In [11]:

def outer_scope():
    var = 1
    def test_fun(input):
        var = 2
        return input + var
    print(f"original var: {var}\noutput of function: {test_fun(1)}")
outer_scope()

original var: 1
output of function: 3


## Impose keywords arguments

For those arguments that are really required to by keyword based, you can enforce it.

In [12]:
def fun(a, b=None, c=True, d=False):
    return a, b, c, d

def fun2(a, b=None, *, c=True, d=False):
    return a, b, c, d

fun(1,2,3,4)
try:
    fun2(1,2,3,4)
    print("Not here!")
except TypeError:
    fun2(1,2,c=3,d=4)
    print("Here!")

Here!


## Decorator for debugging

Use `functools.wraps` to make sure that you can have the proper `__name__`

In [13]:
from functools import wraps

def trace(func):
    @wraps(func)  # to keep the __name__ as func.__name__ and not "wrapper"
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

@trace
def function(a):
    if a < 0:
        return 0
    return function(a-1) + 3


function(10)

function((-1,), {}) -> 0
function((0,), {}) -> 3
function((1,), {}) -> 6
function((2,), {}) -> 9
function((3,), {}) -> 12
function((4,), {}) -> 15
function((5,), {}) -> 18
function((6,), {}) -> 21
function((7,), {}) -> 24
function((8,), {}) -> 27
function((9,), {}) -> 30
function((10,), {}) -> 33


33

In [14]:
help(function)  # notice that now the name is function and not wrapper

Help on function function in module __main__:

function(a)



## Iterators

Use them instead of storing the results in a list to spare memory!

Use the `__iter__` method to define the way to iterate over your data.

Use **generator expressions** to be lighter on memory!

Compose generators with `yield from`.

Use `itertools` functions, as they are very effective on iterators.

In [15]:
from collections.abc import Iterator

class IterableFoo:
    """Minimal iterable class.
    
    Args:
        lista (iterator):
            the list of inputs
    """
    def __init__(self, lista):
        self.lista = lista

    def __iter__(self):  # this is what is called when doing for x in X:
        for item in self.lista:
            yield item*4


my_list = IterableFoo([0, 1, 2, 3])
for i in my_list:
    print(i)

# define a generator
def generator(a):
    try:
        for item in a:
            yield a+3
    except TypeError:
        yield 0

# build the generator instance
it = generator([1, 2, 3])

# check whether it is an iterator
assert isinstance(it, Iterator)

not_it = generator(1)
next(not_it)
try: 
    next(not_it)
except StopIteration:
    print("We exhausted the iterator on the first call")
    
# generator expression: a.k.a. comprehension for generators
gen = (i**2 for i in range(1000000000))
print(gen)

# to compose two generators:
def my_generator():
    yield 1
    yield 2
    yield 3

# this gets data from the child() generator
def fast():
    yield from my_generator()
    
fast()

# P.S. do NOT use send, as it is complicated
def child2():
    for i in range(1_000_000):
        ii = yield i
        print("received: ", ii)
        
it = iter(child2())
print(f"First iteration: {next(it)}, sending"
      f" value: {it.send(10)}, further "
      f"iteration: {next(it)}, further iteration: {next(it)}")

# use itertools functions, as they are very effective
from itertools import combinations_with_replacement
it = combinations_with_replacement([1, 2, 3, 4], 2)
print(list(it))

0
4
8
12
We exhausted the iterator on the first call
<generator object <genexpr> at 0x7f7a46ab8ba0>
received:  10
received:  None
received:  None
First iteration: 0, sending value: 1, further iteration: 2, further iteration: 3
[(1, 1), (1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 3), (3, 4), (4, 4)]


## Use hooks to treat corner cases

Hooks are functions passed to the object that can deal with corner cases. Pass hooks to deal with them.

In [16]:
# defaultdict allow for hooks to treat missing keys
from collections import defaultdict

# define the hook to handle missing keys
def missing_key():
    """returns 0 as value for a missing key"""
    return 100

d = defaultdict(missing_key)  # empty dictionary, with the policy for missing keys

d["key1"]
print(d)

defaultdict(<function missing_key at 0x7f7a46ae2b80>, {'key1': 100})


# Use of `@classmethod`, `super()` and mix-in classes

In [17]:
class Mixin:
    """The Mixin class does not need any
    __init__ constructor, as it will not
    have any instance per se but will be 
    used by other class to inherit the methods.
    """
    def method1(self):
        yield 1
        yield 2

    
class Parent(Mixin):  # inherit from Mixin class
    def __init__(self, value):
        self.value = value
        self.__secret_value = value + 1  # private, cannot access

    @classmethod  # this method is bound to the class and not to self
    def method2(cls, value):  # call it withput cls, as that is automatic!
        return cls(value)  # initialise ann instance
        
class Child(Parent):
    def __init__(self, value):
        super().__init__(value)  # initialise the parent class

# initialise the child class
child = Child(10)

# results
print(f"The Child class inherits from the Parent\n class the attribute `value` "
      f"and it is {child.value}.\n The methods from the mixin class are also\n"
      f"available : {next(child.method1())},\n and {child.method2(1)}")

# P.S. Try to AVOID provate attributes
try:
    child.__secret_value
except AttributeError:
    print("Cannot access private attributes")

The Child class inherits from the Parent
 class the attribute `value` and it is 10.
 The methods from the mixin class are also
available : 1,
 and <__main__.Child object at 0x7f7a46adbc10>
Cannot access private attributes


## A custom class that looks like a list

Use the containers of `collections.abc` to help you write all the functions.

In [18]:
from collections.abc import Sequence  # contains all the standard methods of lists, e.g. __iter__

class MyList(Sequence):
    def __init__(self, *args):
        self.args = args
    
    def __getitem__(self, i):
        return self.args[i]
    
    def __len__(self):
        return len(self.args)
    
lst = MyList(1,2,3,4)
print(f"Item in position 1: {lst[1]}, Length: {len(lst)}")

for i in lst:
    print(i)

Item in position 1: 2, Length: 4
1
2
3
4


## The use of `@property` or descriptors

The idea is to directly access the attributes, but to add a logic when modifying them. Also, **the syntax is exactly the same one of an attribute**!!

It is more general to use the `__set__` and `__get__` descriptors.

In [19]:
class Test:
    def __init__(self, value):
        self.value = value
    
    @property # this is the getter method
    def value(self):
        print("in the getter...")
        return self._value  # notice the underscore!
    
    @value.setter  # this is the setter method
    def value(self, new_value):
        print("in the setter...")
        if new_value > 0:
            self._value = new_value  # notice the underscore!
        else:
            raise ValueError("Need positive value")
            
tst = Test(1)  # here we set the value in te __init__, hence I expect to use the setter
print(tst.value)  # we read te value
tst.value = 2  # we set the value
print(tst.value)  # we read the value
try:
    tst.value = -2  # we try to set a negative value
except ValueError:
    print("...as expected!")

in the setter...
in the getter...
1
in the setter...
in the getter...
2
in the setter...
...as expected!


In [20]:
class Test2:
    def __init__(self):
        pass
    
    def __get__(self, instance, instance_type):
        print("in the getter...")
        return self.value

    def __set__(self, instance, new_value):
        print("in the setter...")
        if new_value > 0:
            self.value = new_value
        else:
            raise ValueError("Need positive value")

class Outer():
    grade = Test2()
        
tst2 = Outer()  # here we set the value in te __init__, hence I expect to use the setter
tst2.grade = 2  # we set the value
print(tst2.grade)  # we read the value
try:
    tst2.grade = -2  # we try to set a negative value
except ValueError:
    print("...as expected!")

in the setter...
in the getter...
2
in the setter...
...as expected!


## Dynamic creation of class attributes with `__getattribute__`

Watch out for recursions! Evey time that you access an attribute inside the `__getattribute__` you will call `__getattribute__`! Hence, do not forget to user `super()` to actually get the attribute of the instance.

Recall that within a class instance `counter`, the following operation:

```
counter = Counter()  # initialise
counter.count += 1
```
is equivalent to this:
```
value = getattr(counter, 'count')
result = value + 1
setattr(counter, 'count', result)
```

In [21]:
class LazyRecord:
    def __init__(self):
        self.exists = 5
    
    def __getattribute__(self, name):  # called every time an attribute is accessed
        try:
            value = super().__getattribute__(name)  # check if the attribute exists and avoid infinite recurson
            print(f'* Found {name!r}, returning {value!r}')
            return value
        except AttributeError:
            value = f'Value for {name}'
            setattr(self, name, value)  # setting the attribute name and value
            return value
        
    def __setattr__(self, name, value):  # called every time an attribute is set
        print("setting attributes...")
        super().__setattr__(name, value)
    
ll = LazyRecord()

print(ll.__dict__)
ll.foo  # creating a new attribute!
print(ll.__dict__)
ll.foo  # creating a new attribute!
print(ll.__dict__)


setting attributes...
* Found '__dict__', returning {'exists': 5}
{'exists': 5}
setting attributes...
* Found '__dict__', returning {'exists': 5, 'foo': 'Value for foo'}
{'exists': 5, 'foo': 'Value for foo'}
* Found 'foo', returning 'Value for foo'
* Found '__dict__', returning {'exists': 5, 'foo': 'Value for foo'}
{'exists': 5, 'foo': 'Value for foo'}


## Validate all subclasses using `__init_subclass__`

Basically, in the parent class, you prepare a method that is fired every time a new class is declared (not instantiated, created!).

You can also call the method `__set_name__` to initialise Parent's class attributes when a class is created.

In [22]:
class Parent:
    def __init_subclass__(cls):
        super().__init_subclass__()
        # check presence of attribute
        print("checking...")
        print(cls)
        if not hasattr(cls, "attr"):
            raise AttributeError(f"Attribute attr not existing")
            
    def __set_name__(self, owner, name):
        """Called on class creation for each descriptor 
        when its containing class is defined, not at
        instance creation!"""
        print("Inside __set_name__...")
        self.name = name  # name of the class instance
        self.internal_name = '_' + name


class ChildOK(Parent):
    attr = 0  # required to pass validation
    def __init__(self, attr):
        self.attr = attr


class Child(Parent):
    attr = 1
    p = Parent()  # here __set_name__ is called


c = Child()
print(f"internal_name: {c.p.internal_name}")

try:
    class ChildBad(Parent):
        ttr2 = 1

except AttributeError:
    print("The validation worked")

checking...
<class '__main__.ChildOK'>
Inside __set_name__...
checking...
<class '__main__.Child'>
internal_name: _p
checking...
<class '__main__.ChildBad'>
The validation worked


## Use class decorators to decorate all the methods of a class

A class decorator is a simple function that receives a class instance as a parameter and returns either a new class or a modified version of the original class.

In [23]:

from functools import wraps

def trace_func(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) -> '
              f'{result!r}')
        return result
    return wrapper

def generator(klass):
    for i, item in enumerate(dir(klass)):
        if i<25 and item not in ("__class__", "__dict__"):
            yield item


def trace(klass):
    for key in dir(klass):
        if (key in generator(klass)) and key[0] == "_":
            value = getattr(klass, key)
            #print(key, value, type(value))
            wrapped = trace_func(value)
            setattr(klass, key, wrapped)
    return klass  # the modified class

@trace
class TraceDict(dict):
    def _test(self, a):
        return a + 1

print("test 1:")
trace_dict = TraceDict([('hi', 1)])
print("test 2:")
trace_dict._test(2)

test 1:
__new__((<class 'TraceDict'>, [('hi', 1)]), {}) -> {}
__init__(({'hi': 1}, [('hi', 1)]), {}) -> None
test 2:
__getattribute__(({'hi': 1}, '_test'), {}) -> <bound method TraceDict._test of {'hi': 1}>


3

## Multi-threading with Lock to prevent race conditions

In [24]:
from threading import Lock, Thread

class LockingCounter:
    def __init__(self):
        self.lock = Lock()
        self.count = 0
    def increment(self, offset):
        with self.lock:  # this allows to do the following operation in one block and cannot be interrupted by other threads
            self.count += offset

how_many = 10**5  # counts per sensor
N_SENSORS = 5  # number of sensors
counter = LockingCounter()  # couter with lock
threads = []  # list to put the threads

def worker(sensor_index, how_many, counter):
    for _ in range(how_many):
        # Read from the sensor
        counter.increment(1)


for i in range(N_SENSORS):
    print(i)  # i-th sensor
    # in a thread, define what function it shall run and its args
    thread = Thread(target=worker,
                    args=(i, how_many, counter))
    # append to the thread list
    threads.append(thread)
    # dispatch the computation
    thread.start()  # Fan out

# Now all threads are running in parallel...
    
# wait for all threads to have finished before moving to the next step!
for thread in threads:
    thread.join()  # Fan in

# Now all threads are over

expected = how_many * N_SENSORS
found = counter.count
print(f'Counter should be {expected}, got {found}')

0
1
2
3
4
Counter should be 500000, got 500000


## `Queue` class for bounding the number of threads

In general, **you still need to lock**!!
The same logic as Redis.

In [25]:
import threading, queue

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        # here we shall put the logic to be done on a thread
        print(f'Finished {item}')
        q.task_done()

# turn-on the worker thread
threading.Thread(target=worker, daemon=True).start()

# send thirty task requests to the worker
for item in range(30):
    q.put(item)  # Fan out
print('All task requests sent\n', end='')

# block until all tasks are done
q.join()  # Fan in
print('All work completed')

All task requests sent
Finished 0
Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
All work completed


## Use `ThreadPoolExecutor` to do multi-threading 
...and limit the number of threads

In [26]:
from concurrent.futures import ThreadPoolExecutor
import time

MAX_WORK = 8
futures = []

def fun_per_thread(param1, param2):
    time.sleep(param1)
    return param2

tic = time.time()
with ThreadPoolExecutor(max_workers=MAX_WORK) as executor:
    for i in range(MAX_WORK*3):
        future = executor.submit(fun_per_thread, 1, (15-i)*2)
        futures.append(future)  # Fan out

    # now the processes are all going in parallel up to MAX_WORK workers

    for future in futures:
        # this waits for the 
        res = future.result()  # Fan in
        # print(res)
toc = time.time()
# Expectig the time to be around 3 seconds
print(toc-tic)

3.0076470375061035


## Use `ProcessPoolExecutor` for multi CPU

Actually, in each CPU a new python interpreter is spawned

In [27]:
try:
    from concurrent.futures import ProcessPoolExecutor
    import time

    MAX_WORK = 2
    futures = []

    def fun_per_thread(param1, param2):
        time.sleep(param1)
        return param2

    tic = time.time()
    with ProcessPoolExecutor(max_workers=MAX_WORK) as executor:  # run in a special context with with statement
        for i in range(MAX_WORK*3):
            future = executor.submit(fun_per_thread, 1, (15-i)*2)
            futures.append(future)  # Fan out

        # now the processes are all going in parallel up to MAX_WORK workers

        for future in futures:
            # this waits for the 
            res = future.result()  # Fan in
            # print(res)
    toc = time.time()
    # Expectig the time to be around 3 seconds
    print(toc-tic)
except:
    print("Canot work on interactive interpreter!")

Canot work on interactive interpreter!


Process SpawnProcess-1:
Traceback (most recent call last):
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'fun_per_thread' on <module '__main__' (built-in)>
Process SpawnProcess-2:
Traceback (most recent call last):
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/multiprocessing/process.py", line 108, in ru

## Use `with` for special contexts

In [28]:
# this is very pythonic
var = ""
with open('effective_python.ipynb', 'r') as handle:
    var = handle.read()
# the file is closed automatically

## use `Decimal` and `Fraction` when doing symbolic operations

... and when precision is imporant!

In [29]:
from decimal import Decimal, ROUND_UP
from fractions import Fraction

a = Decimal("1.46")
b = 2*a
print(b)
bb = b.quantize(Decimal("0.1"), rounding=ROUND_UP)
print(bb)
f = Fraction(1,3)
print(f*f)

2.92
3.0
1/9


## profiling in python

In [30]:
from cProfile import Profile
from pstats import Stats  # to do stats on the profiler
from math import gcd

test_list = [[123223521332122222222, 4139790825322222222], 
             [2322, 42343], 
             [67867, 123412323]]

def test():
    for pair in test_list:
        print("computing...")
        for i in range(1000):
            gcd(*pair)

profiler = Profile()
profiler.runcall(test)
stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

computing...
computing...
computing...
         3097 function calls in 0.002 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.002    0.002 399789443.py:9(test)
     3000    0.001    0.000    0.001    0.000 {built-in method math.gcd}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        6    0.000    0.000    0.000    0.000 iostream.py:500(write)
        7    0.000    0.000    0.000    0.000 iostream.py:206(schedule)
        7    0.000    0.000    0.000    0.000 socket.py:474(send)
        7    0.000    0.000    0.000    0.000 threading.py:1113(is_alive)
        6    0.000    0.000    0.000    0.000 iostream.py:437(_schedule_flush)
        7    0.000    0.000    0.000    0.000 threading.py:1059(_wait_for_tstate_lock)
        6    0.000    0.000    0.000    0.000 iostream.py:418(_is_master_process)
        7    0.000    0.000    0.000    0.000 {method 'acquire

## Use the bisection algorithm to look for items in lists

It is very fast!

In [31]:

from bisect import bisect_left

data = range(10**5)

index = bisect_left(data, 91234)     # Exact match
assert index == 91234
#Consider Searching Sorted Sequences with bisect
index = bisect_left(data, 91234.56) # Closest match
assert index == 91235

## Use of garbage collector and `tracemalloc`

In [32]:
import gc
import tracemalloc

found_objects = gc.get_objects()
print('Now:', len(found_objects))

tracemalloc.start(10)
time1 = tracemalloc.take_snapshot()


Now: 72139


## Use of priority lists to avoid sorting

The magic function is called `heappush`.

In general, use **type hints**. If you want to use defined classes as types, do not forget to use
```
from __future__ import annotations
```

In [33]:
from heapq import heappush
from datetime import date
from typing import Callable, List, TypeVar
from __future__ import annotations  # to use defined classes as types!

class Object:
    def __init__(self, name: str, due_date: date) -> None:
        self.name = name
        self.due_date = due_date

    def __lt__(self, other: Object) -> bool:  # this is to define the "<" operation between objects
        return self.due_date < other.due_date
    
    def __repr__(self):  # to have print work
        return f"({self.name}, {self.due_date})"
    
queue = []

# adding objects to queue in order of due date directly!
def add_to_queue(obj: Object) -> None:
    heappush(queue, obj)

for name, due_date in zip(["a", "b", "c"], [date(1990,3,1), 
                                            date(1990,2,2),
                                            date(1989,11,3)]):
    add_to_queue(Object(name, due_date))
# indeed they are sorted by due date
print(queue)

[(c, 1989-11-03), (a, 1990-03-01), (b, 1990-02-02)]


In [34]:
time2 = tracemalloc.take_snapshot()
time2.compare_to(time1, 'lineno')

[<StatisticDiff traceback=<Traceback (<Frame filename='/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/IPython/core/compilerop.py' lineno=101>,)> size=5909 (+5909) count=101 (+101)>,
 <StatisticDiff traceback=<Traceback (<Frame filename='/Users/matteocaorsi/opt/anaconda3/lib/python3.9/codeop.py' lineno=143>,)> size=5258 (+5037) count=72 (+70)>,
 <StatisticDiff traceback=<Traceback (<Frame filename='/Users/matteocaorsi/opt/anaconda3/lib/python3.9/json/decoder.py' lineno=353>,)> size=2789 (+2789) count=25 (+25)>,
 <StatisticDiff traceback=<Traceback (<Frame filename='/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/IPython/core/compilerop.py' lineno=159>,)> size=2588 (+2588) count=28 (+28)>,
 <StatisticDiff traceback=<Traceback (<Frame filename='/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/jupyter_client/session.py' lineno=917>,)> size=2358 (+2358) count=10 (+10)>,
 <StatisticDiff traceback=<Traceback (<Frame filename='/var/folders/y1/3h1x2nq50

In [35]:
stats = time2.compare_to(time1, 'traceback')
top = stats[0]
print('Biggest offender is:')
print('\n'.join(top.traceback.format()))

Biggest offender is:
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 446
    await dispatch(*args)
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 353
    await result
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 648
    reply_content = await reply_content
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 353
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/zmqshell.py", line 533
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2901
    result = self._run_cell(
  File "/Users/matteocaorsi/opt/anaconda3/lib/python3.9/site-packages/IPython/core/int

In [36]:
found_objects = gc.get_objects()
print('And now:', len(found_objects))

And now: 73883


## Logging in python

In [37]:
import logging

# initialise
logging.basicConfig(filename='example.log', encoding='utf-8', level=logging.DEBUG)

lista = list(range(9))

for i in range(10):
    try:
        lista[i]
        logging.info(lista[i])
    except IndexError:
        logging.error(f"The index {i} is out of bounds!")

try:
    final = lista[i]
except IndexError:
    logging.critical("Cannot compute the final result.")